In [2]:
!mkdir -p ~/. kaggle
!cp kaggle.json ~/.kaggle/


cp: cannot stat 'kaggle.json': No such file or directory


In [3]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 99% 1.05G/1.06G [00:06<00:00, 164MB/s]
100% 1.06G/1.06G [00:06<00:00, 167MB/s]


In [4]:
import zipfile
cdfi = zipfile.ZipFile('/content/dogs-vs-cats.zip','r')
cdfi.extractall('/content')
cdfi.close()

In [6]:
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.applications.vgg16 import VGG16

In [16]:
model = VGG16(
    include_top = False,
    weights="imagenet",
    input_shape = (150,150,3)
)

In [17]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [18]:
fcl = Sequential()

fcl.add(model)
fcl.add(Flatten())
fcl.add(Dense(256,activation="relu"))
fcl.add(Dense(1,activation="sigmoid"))

In [19]:
fcl.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 16812353 (64.13 MB)
Trainable params: 16812353 (64.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [41]:

# if we are using feature extraction method
model.trainable = False # i am freezing the convo layer.

# if we are using fine tununing

# model.trainable = True

# set_trainable = False

# for layer in model.layers:
#   if layer.name == 'block5_conv1':
#     set_trainable = True
#     if set_trainable:
#       layer.trainable = True
#     else:
#       layer.trainable = False

# for layer in model.layers:
#   print(layer.name,layer.trainable)

In [28]:
train_ds = keras.utils.image_dataset_from_directory(
    directory = "/content/train",
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 32,
    image_size = (150,150)
)

test_ds = keras.utils.image_dataset_from_directory(
    directory = "/content/test",
    labels = "inferred",
    label_mode = "int",
    batch_size = 32,
    image_size = (150,150)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [30]:
import tensorflow

In [31]:
# Here we will do nor,alize bcoz my pixels are varing betwwen 0-255 i am doing here from 0-1

def process(image,label):
  image = tensorflow.cast(image/255. , tensorflow.float32)
  return image,label

train_ds = train_ds.map(process)
test_ds = test_ds.map(process)

In [38]:
fcl.compile(optimizer="adam",loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
history = fcl.fit(train_ds,epochs=10,validation_data=test_ds)